In [46]:
# 必要なライブラリをインストール
! pip install requests certifi

# ライブラリのインポート
import pandas as pd
import requests
import tempfile
import certifi

# セキュリティを維持したままファイルをダウンロード
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())  # SSL証明書の確認を行う
response.raise_for_status()  # ステータスコードをチェックし、問題があれば例外を発生

# 一時ファイルにデータを保存
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name  # 一時ファイルのパスを保持

# pandasでデータを読み込む
pwt90 = pd.read_stata(tmp_path)

# Filter and select relevant columns
data = pwt90.loc[pwt90['country'].isin(['United Kingdom'])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [47]:
# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['hours'] = data['emp'] * data['avh']  # L
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha))
data['lab_term'] = data['hours'] / data['pop']  # L/N
data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
    alpha=1 - x['labsh'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
)).reset_index(drop=True).dropna()

# Print first 10 rows of data
print(data.head(3))

   year countrycode       rgdpna        rkna        pop        emp  \
0  1960         GBR  671028.2500  2813235.25  52.352610  23.774094   
1  1961         GBR  685483.8125  2913184.50  52.787446  24.049974   
2  1962         GBR  697548.9375  2997102.75  53.272267  24.182936   

           avh     labsh    rtfpna     alpha           y_n         hours  \
0  2208.754511  0.603153  0.591720  0.396847  28225.187500  52511.136539   
1  2173.661236  0.603153  0.594753  0.396847  28502.476562  52276.497160   
2  2156.257459  0.603153  0.596786  0.396847  28844.675781  52144.635511   

   tfp_term  cap_term     lab_term  y_n_shifted  tfp_term_shifted  \
0  0.418967  2.567757  1003.028059   100.000000        100.000000   
1  0.422533  2.590834   990.320632   100.982422        100.851234   
2  0.424930  2.609570   978.832668   102.194801        101.423439   

   cap_term_shifted  lab_term_shifted  
0         99.999992        100.000000  
1        100.898712         98.733094  
2        101.6283

/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_51932/1162399826.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(


In [65]:
# イギリス
data = pwt90.loc[pwt90['country'].isin(['United Kingdom']) & (pwt90['year'] >= 1960) & (pwt90['year'] <= 2000)]
data = data[['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']].dropna()

# 追加変数の計算
data['gdp_growth'] = data['rgdpna'].pct_change() * 100  # GDP成長率
data['tfp_growth'] = data['rtfpna'].pct_change() * 100  # TFP成長率
data['capital_deepening'] = ((data['rkna'] / data['emp']).pct_change()) * 100  # 資本深化
data['alpha'] = 1 - data['labsh']  # 労働のシェアから資本のシェアを計算
data['tfp_share'] = data['rtfpna'] / data['rgdpna']  # TFPシェア
data['capital_share'] = data['rkna'] / data['rgdpna']  # 資本シェア


In [67]:
# 結果の表示
print(data[['year', 'gdp_growth', 'tfp_growth', 'capital_deepening', 'tfp_share', 'capital_share']])


      year  gdp_growth  tfp_growth  capital_deepening     tfp_share  \
3975  1960         NaN         NaN                NaN  8.818114e-07   
3976  1961    2.154243    0.512564           2.364945  8.676402e-07   
3977  1962    1.760089    0.341833           2.314985  8.555476e-07   
3978  1963    3.867686    1.798344           2.667916  8.385027e-07   
3979  1964    5.614042    2.375484           3.295004  8.127907e-07   
3980  1965    2.325571    0.455630           3.206193  7.979374e-07   
3981  1966    1.948059    0.740147           3.651047  7.884832e-07   
3982  1967    2.229297    1.393485           5.387759  7.820367e-07   
3983  1968    7.165110    5.421209           4.944766  7.693105e-07   
3984  1969   -0.872093   -2.729756           3.959727  7.548936e-07   
3985  1970    2.052784    1.497304           3.981853  7.507847e-07   
3986  1971    3.481662    2.931643           3.805602  7.467941e-07   
3987  1972    4.227173    2.838159           2.990818  7.368417e-07   
3988  